In [2]:
from ollama import chat
import os
import time

# For CUDA GPUs:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # usually 0 is your RTX GPU

import torch
print(torch.cuda.get_device_name(0))  # confirm it's RTX 3050

NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
def simplifier(task):
    prompt = f"""
You are a drone command simplifier.

Your job is to rewrite user instructions into unambiguous, structured English using the following rules:

### Format:
- Do **not** output any text other than the instructions.
- Do **not** include explanations
- Use precise, clean English steps
- **Only one movement**  per line
- Use the format: "Drone X: Action Y" or "Both drones: Action Y"

### Rules:
- Keep the original order of events exactly.
- Do not group or reorder.
- Include all actions, including takeoff and landing.


### Your task:

Input: "{task}"
Output:
"""
    response = chat(
        model="qwen3:4b",
        messages=[{"role": "user", "content": prompt}],
        stream=False
    )
    full_output = response["message"]["content"]
    if "<think>" in full_output:
        cleaned_output = full_output.split("</think>")[-1].strip()
    else:
        cleaned_output = full_output.strip()
    print(f"Simplified task: {cleaned_output}\n==========================")
    return cleaned_output

In [4]:
def GSCE(model, task):
    messages = [ 
        {
            "role": "system",
            "content": """
**Guidelines**
Write a Python script for controlling multiple drones using AirSim.
Break tasks into steps and return only the script—no additional information.
Do not define functions unless explicitly requested.
Comment each step in the code as it is in the example.

**APIs**
aw.fly_to([x, y, z], drone_id): Move the specified drone (drone_id) to XYZ.
aw.get_drone_position(drone_id): Get current position [x, y, z] of the the specified drone (drone_id).
aw.get_yaw(drone_id): Get the current yaw in degrees of the specified drone (drone_id).
aw.set_yaw(yaw_deg, drone_id): Set the yaw angle in degrees of the specified drone (drone_id).
aw.takeoff(drone_id): Take off and hover with the specified drone (drone_id).
aw.land(drone_id): Land at current position with the specified drone (drone_id).

**Constraints**
XYZ coordinate system:
+X = North
+Y = East
+Z = Up

Yaw convention:
0° = North
90° = East
180° = South
270° = West
Clockwise turn = positive yaw
Counter-clockwise turn = negative yaw
Normalize yaw to [0°, 360°)
Initial yaw = 0°

Diagonal directions:
North-East = 45°
South-East = 135°
South-West = 225°
North-West = 315°

**Example**
 
Query: "Drone 1: takeoff  
Drone 1: face North-East
Drone 1: fly 3 meters forward
Drone 1: fly up 8 meters  
Drone 2: takeoff  
Drone 2: face South-East
Drone 1: fly 4 meters  forward 
Drone 1: turn 90° clockwise
Drone 1: fly 2 meters forward
Both drones: fly down 1 meter  
Both drones: fly 2 meters forward
Drone 2: land  
Drone 1: land"

Answer:  

import math

## Step: Drone 1 - Take off
aw.takeoff(drone_id=1)

## Step: Drone 1 - Face North-East (45°)
yaw_1 = 45
aw.set_yaw(yaw_1, drone_id=1)

## Step: Drone 1 - Fly 3 meters forward
current_position_1 = aw.get_drone_position(drone_id=1)
distance_1 = 3
theta_1 = math.radians(yaw_1)
dx_1 = distance_1 * math.cos(theta_1)
dy_1 = distance_1 * math.sin(theta_1)
aw.fly_to([current_position_1[0] + dx_1, current_position_1[1] + dy_1, current_position_1[2]], drone_id=1)

## Step: Drone 1 - Fly up 8 meters (Up = +Z)
current_position_1 = aw.get_drone_position(drone_id=1)
aw.fly_to([current_position_1[0], current_position_1[1], current_position_1[2] + 8], drone_id=1)

## Step: Drone 2 - Take off
aw.takeoff(drone_id=2)

## Step: Drone 2 - Face South-East (135°)
yaw_2 = 135
aw.set_yaw(yaw_2, drone_id=2)

## Step: Drone 2 - Fly 4 meters forward
current_position_2 = aw.get_drone_position(drone_id=2)
distance_2 = 4
theta_2 = math.radians(yaw_2)
dx_2 = distance_2 * math.cos(theta_2)
dy_2 = distance_2 * math.sin(theta_2)
aw.fly_to([current_position_2[0] + dx_2, current_position_2[1] + dy_2, current_position_2[2]], drone_id=2)

## Step: Drone 1 - Turn 90° clockwise (positive turn)
yaw_1 = aw.get_yaw(drone_id=1)
yaw_1 = (yaw_1 + 90) % 360
aw.set_yaw(yaw_1, drone_id=1)

## Step: Drone 1 - Fly 2 meters forward
current_position_1 = aw.get_drone_position(drone_id=1)
distance_1 = 2
theta_1 = math.radians(yaw_1)
dx_1 = distance_1 * math.cos(theta_1)
dy_1 = distance_1 * math.sin(theta_1)
aw.fly_to([current_position_1[0] + dx_1, current_position_1[1] + dy_1, current_position_1[2]], drone_id=1)

## Step: Both drones - Fly down 1 meter (Down = -Z)
current_position_1 = aw.get_drone_position(drone_id=1)
current_position_2 = aw.get_drone_position(drone_id=2)
aw.fly_to([current_position_1[0], current_position_1[1], current_position_1[2] - 1], drone_id=1)
aw.fly_to([current_position_2[0], current_position_2[1], current_position_2[2] - 1], drone_id=2)

## Step: Both drones - Fly forward 2 meters
current_position_1 = aw.get_drone_position(drone_id=1)
current_position_2 = aw.get_drone_position(drone_id=2)
distance = 2
theta_1 = math.radians(yaw_1)
theta_2 = math.radians(yaw_2)
dx_1 = distance * math.cos(theta_1)
dy_1 = distance * math.sin(theta_1)
dx_2 = distance * math.cos(theta_2)
dy_2 = distance * math.sin(theta_2)
aw.fly_to([current_position_1[0] + dx_1, current_position_1[1] + dy_1, current_position_1[2]], drone_id=1)
aw.fly_to([current_position_2[0] + dx_2, current_position_2[1] + dy_2, current_position_2[2]], drone_id=2)

## Step: Land Drone 2
aw.land(drone_id=2)

## Step: Land Drone 1
aw.land(drone_id=1)
"""
        },
        {"role": "user",
         "content": f"""
**Your task**  
Query: "{task}"

Answer:
"""}]


    response = chat(
        model=model,
        messages=messages,
        stream=False
    )

    return response["message"]["content"]

In [5]:
t1 = """Both drones: Take off  
Drone 1: Face North-West  
Drone 1: Fly 1 meter  
Drone 2: Face South-West  
Drone 2: Fly 5 meters  
Both drones: Face North  
Both drones: Fly 3 meters forward  
Drone 2: Land  
Drone 1: Land"""

t2 = """"Drone 1: Take off  
Drone 1: Face North-East  
Drone 1: Fly 3 meters  
Drone 2: Take off  
Drone 2: Face South-East  
Drone 2: Fly 4 meters  
Drone 1: Turn 25° clockwise  
Drone 1: Fly 2 meters  
Both drones: Go up 1 meter  
Drone 2: Land  
Drone 1: Land"""

t3 = """Both drones: take off
Both drones: fly up 2 meters  
Drone 2: fly down 1 meter  
Drone 1: Face East
Drone 1: fly 3 meters  
Drone 1: land  
Drone 2: turn 20 degrees counter-clockwise  
Drone 2: fly 3 meters  
Drone 2: land"""

t4 = "Both drones take off, Drone 1 turns North-West and flies 1 meters, Drone 2 turns South-West and flies 5 meters, both turn North and fly 3 meters forward, Drone 2 lands first, then Drone 1."

t5 = "Drone 1 takes off, turns North-East, flies 3 meters. Then, Drone 2 takes off, turns South-East, flies 4 meters. Then Drone 1 turns 25° clockwise and flies 2 meters. Both drones go up 1 meter. Then, Drone 2 lands, then Drone 1 lands."

t6 = "Both drones takes off and go up 2 meters, Drone 2 goes down 1 meter, Drone 1 turns East and flies 3 meters, lands, then Drone 2 turns 20 degrees counter-clockwise, flies 3 meters, and lands."


m1 = "qwen2.5-coder:1.5b"
m2 = "qwen2.5-coder:3b"
m3 = "codegemma:7b"

In [7]:
tasks = [("Task 1", t1), ("Task 2", t2), ("Task 3", t3)]

for name, task in tasks:
    for i in range(2):
        start = time.time()
        result = GSCE(model=m2, task=task)
        end = time.time()
        print(f"{name} - Run {i+1}:\n{result}\n--- Inference Time: {end - start:.2f} seconds ---\n" + "="*90)

Task 1 - Run 1:
```python
import math

## Step: Both drones - Take off
aw.takeoff(drone_id=1)
aw.takeoff(drone_id=2)

## Step: Drone 1 - Face North-West (315°)
yaw_1 = 315
aw.set_yaw(yaw_1, drone_id=1)

## Step: Drone 1 - Fly 1 meter
current_position_1 = aw.get_drone_position(drone_id=1)
distance_1 = 1
theta_1 = math.radians(yaw_1)
dx_1 = distance_1 * math.cos(theta_1)
dy_1 = distance_1 * math.sin(theta_1)
aw.fly_to([current_position_1[0] + dx_1, current_position_1[1] + dy_1, current_position_1[2]], drone_id=1)

## Step: Drone 2 - Face South-West (225°)
yaw_2 = 225
aw.set_yaw(yaw_2, drone_id=2)

## Step: Drone 2 - Fly 5 meters
current_position_2 = aw.get_drone_position(drone_id=2)
distance_2 = 5
theta_2 = math.radians(yaw_2)
dx_2 = distance_2 * math.cos(theta_2)
dy_2 = distance_2 * math.sin(theta_2)
aw.fly_to([current_position_2[0] + dx_2, current_position_2[1] + dy_2, current_position_2[2]], drone_id=2)

## Step: Both drones - Face North (90°)
yaw_1 = 90
yaw_2 = 90
aw.set_yaw(yaw_1, d